In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import os
import time
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast

In [25]:
# Data Loading and Preprocessing with class weighting
def load_data(data_dir, batch_size, num_workers, pin_memory):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize all images to 224x224
        transforms.ToTensor(),  # Convert images to PyTorch tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
    ])

    dataset = ImageFolder(root=data_dir, transform=transform)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    # Compute class weights
    class_counts = np.zeros(len(dataset.classes))
    for _, index in train_dataset:
        class_counts[index] += 1
    class_weights = 1.0 / (class_counts + 1e-6)  # Adding a small constant to avoid division by zero
    sample_weights = [class_weights[label] for _, label in train_dataset]

    # Setup the WeightedRandomSampler
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=num_workers, pin_memory=pin_memory)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory)
    return train_loader, test_loader



In [26]:
# Model Definition
def define_model(num_classes):
    model = resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes),
        nn.LogSoftmax(dim=1)
    )
    return model

In [27]:
# Training the model with timing
def train_model(model, train_loader, criterion, optimizer, device, epochs):
    model.train()
    scaler = GradScaler()
    for epoch in range(epochs):
        start_time = time.time()  # Start time for the epoch
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            if (i + 1) % 20 == 0:
                print(f'Epoch {epoch + 1}, Step {i + 1}, Loss: {running_loss / 20:.4f}')
                running_loss = 0.0

        end_time = time.time()  # End time for the epoch
        epoch_duration = end_time - start_time  # Calculate duration of the epoch
        print(f'Epoch {epoch + 1} completed in {epoch_duration:.2f} seconds')



In [28]:
# Evaluate the model
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy:.2f}%')


In [29]:
# Main function
def main():
    data_dir = '/content/drive/MyDrive/JPEGImages'
    batch_size = 64
    num_workers = 8
    pin_memory = True
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    train_loader, test_loader = load_data(data_dir, batch_size, num_workers, pin_memory)
    model = define_model(50)  # Assuming 50 animal classes
    model.to(device)

    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_model(model, train_loader, criterion, optimizer, device, 10)
    evaluate_model(model, test_loader, device)

if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Step 20, Loss: 3.5018
Epoch 1, Step 40, Loss: 2.9120
Epoch 1, Step 60, Loss: 2.7379
Epoch 1, Step 80, Loss: 2.4131
Epoch 1, Step 100, Loss: 2.3685
Epoch 1, Step 120, Loss: 2.3184
Epoch 1 completed in 28.57 seconds
Epoch 2, Step 20, Loss: 2.2267
Epoch 2, Step 40, Loss: 2.1365
Epoch 2, Step 60, Loss: 1.8330
Epoch 2, Step 80, Loss: 1.9202
Epoch 2, Step 100, Loss: 1.7747
Epoch 2, Step 120, Loss: 1.6653
Epoch 2 completed in 28.34 seconds
Epoch 3, Step 20, Loss: 1.6979
Epoch 3, Step 40, Loss: 1.5737
Epoch 3, Step 60, Loss: 1.4730
Epoch 3, Step 80, Loss: 1.4452
Epoch 3, Step 100, Loss: 1.4008
Epoch 3, Step 120, Loss: 1.3102
Epoch 3 completed in 28.42 seconds
Epoch 4, Step 20, Loss: 1.3526
Epoch 4, Step 40, Loss: 1.2426
Epoch 4, Step 60, Loss: 1.1618
Epoch 4, Step 80, Loss: 1.2531
Epoch 4, Step 100, Loss: 1.0643
Epoch 4, Step 120, Loss: 1.0931
Epoch 4 completed in 28.40 seconds
Epoch 5, Step 20, Loss: 1.1722
Epoch 5, Step 40, Loss: 1.0224
Epoch 5, Step 60, Loss: 0.9775
Epoch 5, Step 8